# Libraries 

In [1]:
%run "/home/cesar/Python_NBs/HDL_Project/Mini HDL/Baseline_ML_Pollution_Concentration_MMA/global_fv.ipynb"

User information is ready!


In [2]:
import plotly.express as px
import seaborn as sns
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

ImportError: /home/cesar/anaconda3/envs/hdl/bin/../lib/libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /home/cesar/anaconda3/envs/hdl/lib/python3.8/site-packages/scipy/fft/_pocketfft/pypocketfft.cpython-38-x86_64-linux-gnu.so)

# User-Defined Functions

In [ ]:
def melted_data(data, col_name): 
    features = list(data.columns)
    data = data.reset_index(level=0)    
    data = pd.melt(data, id_vars =['datetime'], value_vars =features)
    data_melt = data.rename(columns = {"variable":col_name})
    return data_melt

In [ ]:
def data_vis(data, col_name):
    data = melted_data(data, col_name)
    fig = px.line(data, x='datetime', y="value", color=col_name)
    
    fig.show()

In [ ]:
def dtw_analysis(data):
    results = []
    cols = list(data.columns)
    n_cols = len(cols)
    
    for i in cols:
        for j in cols:
            distance, _ = fastdtw(data[i], data[j], dist=euclidean)
            results.append(distance)

    results = np.array(results)
    results = results.reshape((n_cols,n_cols))

    results = pd.DataFrame(results, columns = cols)
    results[" "] = cols
    results = results.set_index(" ")
    return results

In [ ]:
def trend_seasonality_df(data):
    count = 0
    for col in list(data.columns):

        decompose_result_mult = seasonal_decompose(data[[col]])

        trend = decompose_result_mult.trend
        seasonal = decompose_result_mult.seasonal

        if count == 0:
            trend_df = pd.DataFrame([trend])
        else:    
            trend_df = trend_df.append(trend, ignore_index=True)

        if count == 0:
            seasonal_df = pd.DataFrame([seasonal])
        else:    
            seasonal_df = seasonal_df.append(seasonal, ignore_index=True)        

        count+=1

    trend_df = trend_df.transpose()
    trend_df.set_axis(list(data.columns), axis=1,inplace=True)

    seasonal_df = seasonal_df.transpose()
    seasonal_df.set_axis(list(data.columns), axis=1,inplace=True)
    
    return trend_df, seasonal_df

In [ ]:
def adf_test(data, title = 'Metrics'):
    """
    Augmented Dickey-Fuller test for stationarity
    * p-value > 0.05: Data is non-stationary. Fail to reject the null hypothesis (H0). 
    * p-value <= 0.05: Data is stationary. Reject the null hypothesis (H0).
    """
    
    adf = pd.DataFrame({title:[], "p-value":[], "Stationarity":[]})

    for x in data.columns:
        result = adfuller(data[x])
        p_val = result[1]
        txt = "Non-Stationary" if p_val > 0.05 else "Stationary"
        tmp = pd.DataFrame({title:[x], "p-value":[p_val], "Stationarity":[txt]})
        adf = adf.append(tmp, ignore_index=True)
        adf = adf.sort_values("p-value", ascending = False)
        
    return adf.reset_index(drop = True)

# Data

In [ ]:
sql_table = "sima_station_CE"
target = "pm25"

# Define columns of interest from sql table
#     Select all columns:
column = "*"
#     Select specific columns:
#column = "datetime, prs, rainf, rh, sr, tout, wdr, wsr, " + str(target)

# Filter data with WHERE command
sql_where = "where datetime >= \'2021-03-01\'"

# Number of time steps per sample
n_steps = 24

In [ ]:
# Initialize class to create multivariate samples:
multi_ts = multivariate_samples(sql_table, column, sql_where)
data = multi_ts.initial_df()

In [ ]:
# Quick fix

# 1 ------------------------------------------------------------------------------------------------
# Observations:
# - Zero pressure exists only in a perfect vacuum, and outer space is the only place where this occurs naturally. 
#   This means that these are missing measurements
#   By context, it can be substituted by its surrounding mean values
array_1 = data["prs"]
data["prs"] = np.where(array_1 == 0, array_1[array_1>0].mean(), array_1)


In [ ]:
data_dt_index = data.copy()
data = data.reset_index()

In [ ]:
data_vis(data, "Time Series")

# Correlation

In [ ]:
# Spearman correlation is applied if the correlation is thought of as non-linear.
spearman_matrix = data.drop("rainf", axis = 1).corr(method = "spearman")

# Hierarchical clustering for the correlation matrix, where similar time series are placed closer to one another.
sns.clustermap(spearman_matrix)
spearman_matrix

## `* Which of them move similarly?`
Dynamic Time Warping (DTW) to measure similarity between time series, which may vary in speed.

In [ ]:
# A matrix with the similarity distance is calculated for every feature.
# The lower the value, the more similar they are.
# Obviously, a feature compared with itself will have the lowest value (0).

results = dtw_analysis(data_dt_index)
results

# Distance time warping analysis (DTW) recognized the minimum distance between SO_2 and PM10
# PM2.5 has minimum distance with NOx

In [ ]:
results[results>0].min()

# Trend and seasonality analysis

In [ ]:
trend_df, seasonal_df = trend_seasonality_df(data_dt_index.asfreq('H'))

In [ ]:
data_vis(trend_df, "Trend")

In [ ]:
data_vis(seasonal_df, "Seasonality")
# It has no seasonality

# Stationarity test

In [ ]:
adf_test(data_dt_index)